In [1]:
%%bash
git branch -v 

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.
  assembias cc90f59 [ahead 1] checking out to make some plots
* master    c82f359 resyncing these bin files.


In [2]:
from pearce.emulator import OriginalRecipe, ExtraCrispy, SpicyBuffalo

In [3]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

In [4]:
training_dir = '/u/ki/swmclau2/des/PearceLHC_wp_z/'
fiducial_params = {'logM0': 12.20, 'logM1': 13.7, 'alpha': 1.02,
                      'logMmin': 12.1, 'f_c': 0.19, 'sigma_logM': 0.46}
rbins = np.array([  0.06309573,   0.12437607,   0.24517359,   0.34422476, 0.48329302, 0.67854546,\
           0.9526807 , 1.33756775,1.8779508 ,   2.6366509 ,   3.70186906,   5.19743987, 7.29722764, \
           10.24533859,  14.38449888,  20.1958975 , 28.35512583,  39.81071706] )
rpoints = (rbins[1:]+rbins[:-1])/2

In [5]:
y_param = 'logMmin'
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233), ('z', 0.5)]
em_params = dict(emulation_point)
try:
    del em_params[y_param]
except KeyError:
    pass

In [7]:
y_param_idx = -1
for idx, p in enumerate(or_params):
    if p.name==y_param:
        y_param_idx = idx
        break
        
y_vals = np.linspace(or_params[y_param_idx].low, or_params[y_param_idx].high, 4)
em_params[y_param] = y_vals

In [8]:
print em_params[y_param]

[ 11.7         11.96666667  12.23333333  12.5       ]


In [ ]:
print em_params

{'logMmin': array([ 11.7       ,  11.96666667,  12.23333333,  12.5       ]), 'f_c': 0.233, 'logM0': 12.0, 'sigma_logM': 0.533, 'logM1': 13.5, 'alpha': 1.083, 'z': 0.5}


In [ ]:
emu1 = OriginalRecipe(training_dir,params=or_params)

In [ ]:
rpoints = np.logspace(np.log10(rbins[1]), np.log10(rbins[-2]), 50)

In [ ]:
log_xi_pred= emu1.emulate_wrt_r(em_params, rpoints)
#log_xi_cov = np.zeros((log_xi_pred.shape[0], log_xi_pred.shape[0]))
#log_xi_err = np.sqrt(np.diag(log_xi_cov))
#log_xi_pred = log_xi_pred.reshape((-1, len(rpoints)))
#log_xi_err = log_xi_err.reshape((-1, len(rpoints)))
log_xi_err = np.zeros_like(log_xi_pred)

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, lxp, lxe,c in zip(y_vals, log_xi_pred, log_xi_err ,colors):

    plt.plot(rpoints, lxp,label = 'z= %.3f'%label, color = c )

    plt.fill_between(rpoints,lxp+lxe,
                     lxp-lxe, color = c, alpha = 0.3)

plt.xlim(xmin = 0.1, xmax = 30)    
plt.ylim(ymin = 0, ymax = 3.5)
plt.title("OR Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{w_p(r_p) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

In [ ]:
em_params

In [ ]:
x = np.array([[[range(10)], [range(10,21)], [range(20, 31)], [range(30,41)]],
             [[range(5,11)], [range(15,26)], [range(25, 36)], [range(35,46)]]])
print x
print x.shape
print x[:, 0]

In [ ]:
x[:, range(x.shape[0]), :] = x[:, range(x.shape[0]-1, -1, -1), :]
print x
print x[:, 0]

In [ ]:
emu2 = ExtraCrispy(training_dir)#no r in params

In [ ]:
emu1.x[:34, -2:]

In [ ]:
print emu2.redshift_bin_centers
print emu2.y[0, :, 0]
print emu2.y[0, 0, :]

In [ ]:
rpoints = emu2.scale_bin_centers
em_params['z'] = np.array([emu2.redshift_bin_centers[0]])

In [ ]:
log_xi_pred = emu2.emulate_wrt_r(em_params,rpoints)
#log_xi_cov = np.zeros((log_xi_pred.shape[0], log_xi_pred.shape[0]))
#log_xi_err = np.sqrt(np.diag(log_xi_cov))
#print log_xi_pred
#log_xi_pred = log_xi_pred.reshape((-1, len(rpoints)))
#log_xi_err = log_xi_err.reshape((-1, len(rpoints)))
log_xi_err = np.zeros(log_xi_pred.shape)

In [ ]:
print log_xi_pred

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, lxp, lxe,c in zip(y_vals, log_xi_pred, log_xi_err ,colors):

    #plt.plot(rpoints, lxp,label = 'z= %.3f'%label, color = c )
    plt.plot(rpoints, lxp,label = 'z= %.3f'%label, color = c )

    plt.fill_between(rpoints,lxp+lxe,
                     lxp-lxe, color = c, alpha = 0.3)

plt.xlim(xmin = 0.1, xmax = 30)    
plt.ylim(ymin = 0, ymax = 3.5)
plt.title("EC Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{w_p(r_p) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

In [ ]:
v2  = emu2.emulators[0][0].predict(emu2.y[:, 0, 0], np.array([[ 12.5,0.533,12.,13.5,1.083 ,0.233 ]]))
v1 = emu1.emulator.predict(emu1.y,np.array([[ 12.5,0.533,12.,13.5,1.083 ,0.233,emu2.redshift_bin_centers[0], emu2.scale_bin_centers[0] ]])) 
print v1
print v2

In [ ]:
or_params = PARAMS[:]
or_params.append(parameter('z', 0.0, 0.5))
#or_params.append(parameter('r', 0, 1)) #95% sure bounds aren't used for r
emu3 = SpicyBuffalo(training_dir, em_param = 'z', params = or_params)#no r in params

In [ ]:
print len(emu3.emulators)
print emu3.em_bin_centers

In [ ]:
print em_params['z']

In [ ]:
log_xi_pred = emu3.emulate_wrt_r(em_params,rpoints)
#log_xi_cov = np.zeros((log_xi_pred.shape[0], log_xi_pred.shape[0]))
#log_xi_err = np.sqrt(np.diag(log_xi_cov))
#print log_xi_pred
#log_xi_pred = log_xi_pred.reshape((-1, len(rpoints)))
#log_xi_err = log_xi_err.reshape((-1, len(rpoints)))
log_xi_err = np.zeros(log_xi_pred.shape)

In [ ]:
print log_xi_pred.shape

In [ ]:
print log_xi_pred[0, :]

In [ ]:
fig = plt.figure(figsize = (10, 10))
for label, lxp, lxe,c in zip(y_vals, log_xi_pred, log_xi_err ,colors):

    #plt.plot(rpoints, lxp,label = 'z= %.3f'%label, color = c )
    plt.plot(rpoints, lxp,label = 'z= %.3f'%label, color = c )

    plt.fill_between(rpoints,lxp+lxe,
                     lxp-lxe, color = c, alpha = 0.3)

plt.xlim(xmin = 0.1, xmax = 30)    
plt.ylim(ymin = 0, ymax = 3.5)
plt.title("SB Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{w_p(r_p) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

In [ ]:
emu1.x.shape

In [ ]:
emu1.x[:34,-2:]

In [ ]:
emu1.y.shape

In [ ]:
emu2.y.shape

In [ ]:
idx1, idx2 = 0,0

In [ ]:
print emu1.x[idx1,-1]
print emu1.x[idx2*len(emu1.x)/2, -2]

In [ ]:
ec_col = emu2.y[:,idx2, idx1 ]
or_col = emu1.y[np.logical_and(emu1.x[:, -1]==emu1.x[idx1,-1], emu1.x[:, -2] == emu1.x[idx2*len(emu1.x)/2, -2] ) ]

In [ ]:
print or_col.shape
print ec_col.shape

In [ ]:
np.all(or_col == ec_col)

In [ ]:
ec_col[:10]

In [ ]:
or_col[:10]

In [ ]:
from copy import deepcopy

In [ ]:
new_y = deepcopy(emu1.y)

In [ ]:
new_y = new_y.reshape((-1, 17))

In [ ]:
print new_y[:20, 0]

In [ ]:
new_y = new_y.reshape((-1, 2, 17), order='F')

In [ ]:
print new_y[:10,idx2,idx1]

In [ ]:
new_y.shape

In [ ]:
print all(new_y[:,idx2,idx1] == or_col)
print all(new_y[:,idx2,idx1] == ec_col)

In [ ]:
new_y[:10,0,0]

In [ ]:
for idx1 in xrange(17):
    for idx2 in xrange(2):
        ec_col = emu2.y[:,idx2, idx1 ]
        or_col = emu1.y[np.logical_and(emu1.x[:, -1]==emu1.x[idx1,-1], emu1.x[:, -2] == emu1.x[idx2*len(emu1.x)/2, -2] ) ]
        new_y = deepcopy(emu1.y)
        new_y = new_y.reshape((-1, 17))
        new_y = new_y.reshape((-1, 2, 17), order='F')
        print idx1, idx2, all(new_y[:,idx2,idx1] == or_col), all(new_y[:,idx2,idx1] == ec_col), all(ec_col == or_col)
        print 

In [ ]:
idx1, idx2 = 0,0

In [ ]:
print emu1.y[np.logical_and(emu1.x[:, -1]==emu1.x[idx1,-1], emu1.x[:, -2] == emu1.x[idx2*len(emu1.x)/2, -2] ) ][:10]
print emu1.x[np.logical_and(emu1.x[:, -1]==emu1.x[idx1,-1], emu1.x[:, -2] == emu1.x[idx2*len(emu1.x)/2, -2] ), 0 ][:10]

In [ ]:
or_col = emu1.y[np.logical_and(emu1.x[:, -1]==emu1.x[idx1,-1], emu1.x[:, -2] == emu1.x[idx2*len(emu1.x)/2, -2] ) ]

In [ ]:
or_x_col = emu1.x[np.logical_and(emu1.x[:, -1]==emu1.x[idx1,-1], emu1.x[:, -2] == emu1.x[idx2*len(emu1.x)/2, -2] ) ]

In [ ]:
counter = 0
for row_x, row_y in zip(emu1.x, emu1.y):
    if not all(row_x == or_x_col[counter]):
        continue
    print row_y, or_col[counter], row_y==or_col[counter]
    counter+=1
    if counter == len(or_x_col):
        break
print counter